In [6]:
# Import dependencies from Pandas, SKLearn, TensorFlow & MatPlotLib
from path import Path
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

import sklearn as skl
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,MinMaxScaler, LabelEncoder
from sklearn.svm import SVC

In [1]:
from sqlalchemy import create_engine

In [2]:
# Defining db info in config var
jdbc_url="jdbc:postgresql://34.67.52.115/team5k"
config = {'user': 'postgres', 
          "password": "team5kteam5k", 
          "driver":"org.postgresql.Driver",
          "location": "34.67.52.115",
          "db": "team5k",
          "port": "5432"}


#postgres://[user]:[password]@[location]:[port]/[database]
create_engine_str = 'postgresql://' + config["user"] + ":" + config["password"] + "@" + config["location"] + ":" + config["port"] + "/" + config["db"]
db = create_engine(create_engine_str)

In [9]:
#Test query
df=pd.read_sql("SELECT * FROM public.fec_donor_az LIMIT 5;",create_engine_str)
df.head()

,id,cmt_id,amndt_ind,rpt_tp,transaction_pgi,image_num,transaction_tp,entity_tp,last_name,first_name,...,employer,occupation,transaction_dt,transaction_amt,other_id,tran_id,file_num,memo_cd,memo_text,sub id
0,141712,c00310490,n,my,None,99020070054,15,None,might,thomas o,...,cableone,None,1999-06-16,1000,None,None,None,None,None,3061920110007090000
1,141713,c00342436,n,my,None,99020081109,15,None,roberts,susan,...,None,None,1999-03-10,1000,None,None,None,None,None,3061920110007110000
2,141714,c00287953,n,my,None,99020080810,15,None,baker,barry,...,sss corporation,None,1999-05-29,1000,None,None,None,None,None,3061920110007140000
3,141715,c00287953,n,my,None,99020080810,15,None,baker,alice,...,arizona mail order company inc,None,1999-05-29,1000,None,None,None,None,None,3061920110007140000
4,141716,c00287953,n,my,None,99020080810,15,None,baker,paul,...,arizona mail order company inc,None,1999-05-29,1000,None,None,None,None,None,3061920110007140000


In [11]:
#Union all donation tables
contrib_df=pd.read_sql_query('''SELECT id, cmt_id, amndt_ind, rpt_tp, transaction_pgi, image_num, transaction_tp, entity_tp, last_name, first_name, city, state, zip, employer, occupation, transaction_dt, transaction_amt, other_id, tran_id, file_num, memo_cd, memo_text, "sub id"
	FROM public.fec_donor_az
union
SELECT id, cmt_id, amndt_ind, rpt_tp, transaction_pgi, image_num, cast(transaction_tp as text), entity_tp, last_name, first_name, city, state, zip, employer, occupation, transaction_dt, transaction_amt, other_id, tran_id, file_num, memo_cd, memo_text, "sub id"
	FROM public.fec_donor_fl
union
SELECT id, cmt_id, amndt_ind, rpt_tp, transaction_pgi, image_num, transaction_tp, entity_tp, last_name, first_name, city, state, zip, employer, occupation, transaction_dt, transaction_amt, other_id, tran_id, file_num, memo_cd, memo_text, "sub id"
	FROM public.fec_donor_mi
union
SELECT id, cmt_id, amndt_ind, rpt_tp, transaction_pgi, image_num, cast(transaction_tp as text), entity_tp, last_name, first_name, city, state, zip, employer, occupation, transaction_dt, transaction_amt, other_id, tran_id, file_num, memo_cd, memo_text, "sub id"
	FROM public.fec_donor_nc
union
SELECT id, cmt_id, amndt_ind, rpt_tp, transaction_pgi, image_num, transaction_tp, entity_tp, last_name, first_name, city, state, zip, employer, occupation, transaction_dt, transaction_amt, other_id, tran_id, file_num, memo_cd, memo_text, "sub id"
	FROM public.fec_donor_pa
union
SELECT id, cmt_id, amndt_ind, rpt_tp, transaction_pgi, image_num, transaction_tp, entity_tp, last_name, first_name, city, state, zip, employer, occupation, transaction_dt, transaction_amt, other_id, tran_id, cast(file_num as text), memo_cd, memo_text, "sub id"
	FROM public.fec_donor_wi''', create_engine_str)
contrib_df.head()

,id,cmt_id,amndt_ind,rpt_tp,transaction_pgi,image_num,transaction_tp,entity_tp,last_name,first_name,...,employer,occupation,transaction_dt,transaction_amt,other_id,tran_id,file_num,memo_cd,memo_text,sub id
0,374005,c00305342,n,ye,None,20035140709,15,None,stoltzfus,myron,...,None,None,1999-09-17,250,None,None,None,None,None,3061920110006920000
1,513944,c00305342,n,ye,None,20035140739,15,None,galbraith,john w,...,retired,None,1999-12-09,750,None,None,None,None,None,3061920110006980000
2,374077,c00343376,n,ye,None,20035230998,15,None,adezio,ron,...,gmr telecommunication,None,1999-12-17,500,None,None,None,None,None,3061920110006930000
3,240317,c00335729,n,ye,None,20035081187,15,None,zichichi,rose marie,...,ti-coating,None,1999-12-30,500,None,None,None,None,None,3061920110006610000
4,513965,c00331256,a,my,None,99034673585,15,None,bockorny,jill,...,housewife,None,1999-06-14,1000,None,None,None,None,None,3061920110007030000


In [14]:
#See Data Types
contrib_df.dtypes

id                   int64
cmt_id              object
amndt_ind           object
rpt_tp              object
transaction_pgi     object
image_num            int64
transaction_tp      object
entity_tp           object
last_name           object
first_name          object
city                object
state               object
zip                float64
employer            object
occupation          object
transaction_dt      object
transaction_amt      int64
other_id            object
tran_id             object
file_num            object
memo_cd             object
memo_text           object
sub id               int64
dtype: object

In [30]:
contrib_df.count()

id                 600
cmt_id             600
amndt_ind          600
rpt_tp             600
transaction_pgi     13
image_num          600
transaction_tp     600
entity_tp           89
last_name          600
first_name         600
city               599
state              600
zip                597
employer           540
occupation          91
transaction_dt     600
transaction_amt    600
other_id             1
tran_id             89
file_num            89
memo_cd              0
memo_text            2
sub id             600
dtype: int64

In [20]:
#Get Postal Code Info
zip_df=pd.read_sql("SELECT * FROM public.postal_codes;",create_engine_str)
zip_df.set_index('zip', inplace=True)
zip_df.head()

,county,state,stcountyfp,classfp
zip,,,,
36003,Autauga County,AL,1001,H1
36006,Autauga County,AL,1001,H1
36067,Autauga County,AL,1001,H1
36066,Autauga County,AL,1001,H1
36703,Autauga County,AL,1001,H1


In [21]:
zip_df.dtypes

county        object
state         object
stcountyfp     int64
classfp       object
dtype: object

In [36]:
contrib_county_df=contrib_df.join(zip_df, on='zip', how="left", lsuffix="_contrib", rsuffix="_postal")
contrib_county_df.head(50)
contrib_county_df.to_csv("test.csv")

In [33]:
contrib_county_df.dtypes

county              object
state_contrib       object
stcountyfp         float64
classfp             object
id                   int64
cmt_id              object
amndt_ind           object
rpt_tp              object
transaction_pgi     object
image_num            int64
transaction_tp      object
entity_tp           object
last_name           object
first_name          object
city                object
state_postal        object
zip                float64
employer            object
occupation          object
transaction_dt      object
transaction_amt      int64
other_id            object
tran_id             object
file_num            object
memo_cd             object
memo_text           object
sub id               int64
dtype: object

In [68]:
#Get County Votes
votes_df=pd.read_sql("SELECT * FROM public.pres_votes_6t;",create_engine_str)
votes_df['county'] = votes_df['county'] + " County"
votes_df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Arizona,AZ,Apache County,4001,President,Al Gore,democrat,13025,19456,20191203
1,2000,Arizona,AZ,Apache County,4001,President,George W. Bush,republican,5947,19456,20191203
2,2000,Arizona,AZ,Apache County,4001,President,Ralph Nader,green,245,19456,20191203
3,2000,Arizona,AZ,Apache County,4001,President,Other,NA,239,19456,20191203
4,2000,Arizona,AZ,Cochise County,4003,President,Al Gore,democrat,13360,33241,20191203


In [107]:
#Profile Data
    
votes_df["candidatevotes"].value_counts()

NA        100
44         11
54         11
192        11
49         10
         ... 
20347       1
110667      1
5311        1
513         1
4876        1
Name: candidatevotes, Length: 4931, dtype: int64

In [108]:
votes_df["candidatevotes"]=votes_df["candidatevotes"].replace({"NA":0})
votes_df["candidatevotes"].value_counts()

0         100
192        11
54         11
44         11
47         10
         ... 
20347       1
110667      1
5311        1
513         1
7328        1
Name: candidatevotes, Length: 4931, dtype: int64

In [109]:
#Group by Year State, county party

vote_summary_df=votes_df.groupby(["year", "state", "state_po", "county","FIPS", "office", "party","candidatevotes"],as_index=False).sum()
vote_summary_df

,year,state,state_po,county,FIPS,office,party,candidatevotes,totalvotes,version
0,2000,Arizona,AZ,Apache County,4001,President,NA,239,19456,20191203
1,2000,Arizona,AZ,Apache County,4001,President,democrat,13025,19456,20191203
2,2000,Arizona,AZ,Apache County,4001,President,green,245,19456,20191203
3,2000,Arizona,AZ,Apache County,4001,President,republican,5947,19456,20191203
4,2000,Arizona,AZ,Cochise County,4003,President,NA,588,33241,20191203
...,...,...,...,...,...,...,...,...,...,...
6459,2016,Wisconsin,WI,Winnebago County,55139,President,democrat,37054,87144,20191203
6460,2016,Wisconsin,WI,Winnebago County,55139,President,republican,43448,87144,20191203
6461,2016,Wisconsin,WI,Wood County,55141,President,NA,2065,37800,20191203
6462,2016,Wisconsin,WI,Wood County,55141,President,democrat,14232,37800,20191203


In [111]:
vote_summary_df['candidatevotes']=vote_summary_df['candidatevotes'].apply(pd.to_numeric)
vote_summary_df.loc[vote_summary_df['party']=='democrat', 'democratic_votes']= vote_summary_df['candidatevotes']
vote_summary_df.loc[vote_summary_df['party']!='democrat', 'democratic_votes']= 0
vote_summary_df.loc[vote_summary_df['party']=='republican', 'republican_votes']= vote_summary_df['candidatevotes']
vote_summary_df.loc[vote_summary_df['party']!='republican', 'republican_votes']= 0
vote_summary_df.loc[(vote_summary_df['party']  =='republican') | (vote_summary_df['party'] =='democrat'), 'other_votes']= 0
vote_summary_df.loc[(vote_summary_df['party']!='republican') & (vote_summary_df['party']!='democrat'), 'other_votes']= vote_summary_df['candidatevotes']
#vote_summary_df['sum_votes']=vote_summary_df['democratic_votes'] + vote_summary_df['republican_votes'] + vote_summary_df['other_votes']
vote_summary_df.head()

,year,state,state_po,county,FIPS,office,party,candidatevotes,totalvotes,version,democratic_votes,republican_votes,other_votes
0,2000,Arizona,AZ,Apache County,4001,President,NA,239,19456,20191203,0.0,0.0,239.0
1,2000,Arizona,AZ,Apache County,4001,President,democrat,13025,19456,20191203,13025.0,0.0,0.0
2,2000,Arizona,AZ,Apache County,4001,President,green,245,19456,20191203,0.0,0.0,245.0
3,2000,Arizona,AZ,Apache County,4001,President,republican,5947,19456,20191203,0.0,5947.0,0.0
4,2000,Arizona,AZ,Cochise County,4003,President,NA,588,33241,20191203,0.0,0.0,588.0


In [113]:
#Profile Data

votes_df["state"].value_counts()

North Carolina    1600
Michigan          1328
Wisconsin         1152
Florida           1072
Pennsylvania      1072
Arizona            240
Name: state, dtype: int64

In [114]:
#Profile Data

votes_df["year"].value_counts()

2000    1616
2016    1212
2012    1212
2008    1212
2004    1212
Name: year, dtype: int64

In [147]:
vote_county_df=vote_summary_df.groupby(["year", "state", "state_po", "county","FIPS"],as_index=False).sum()
vote_county_df.head()

,year,state,state_po,county,FIPS,candidatevotes,totalvotes,version,democratic_votes,republican_votes,other_votes
0,2000,Arizona,AZ,Apache County,4001,19456,77824,80764812,13025.0,5947.0,484.0
1,2000,Arizona,AZ,Cochise County,4003,33241,132964,80764812,13360.0,18180.0,1701.0
2,2000,Arizona,AZ,Coconino County,4005,40883,163532,80764812,20280.0,17562.0,3041.0
3,2000,Arizona,AZ,Gila County,4007,17736,70944,80764812,7700.0,9158.0,878.0
4,2000,Arizona,AZ,Graham County,4009,9664,38656,80764812,3355.0,6007.0,302.0


In [154]:
#Define county winner
vote_county_df.loc[vote_county_df['democratic_votes']>vote_county_df['republican_votes'], "winner"]='D'
vote_county_df.loc[vote_county_df['democratic_votes']<vote_county_df['republican_votes'], "winner"]='R'
vote_county_df['winner'].value_counts()

R    1459
D     561
Name: winner, dtype: int64

In [139]:
#Get Employment Data
unemployment_df=pd.read_sql_query('''select 2000,"FIPStxt",
"Stabr",
"County",
" Unemployed_2000 ",
" Civilian_labor_force_2000 ","POP_COU"
FROM public.unemployment where "Stabr" in ('AZ','MI','WI','FL','PA','NC')
union
select
2004, "FIPStxt",
"Stabr",
"County",
" Unemployed_2004 ",
" Civilian_labor_force_2004 ","POP_COU"
FROM public.unemployment where "Stabr" in ('AZ','MI','WI','FL','PA','NC')
union
select
2008,"FIPStxt",
"Stabr",
"County",
" Unemployed_2008 ",
" Civilian_labor_force_2008 ","POP_COU"
FROM public.unemployment where "Stabr" in ('AZ','MI','WI','FL','PA','NC')
union
select
2012,"FIPStxt",
"Stabr",
"County",
" Unemployed_2012 ",
" Civilian_labor_force_2012 ","POP_COU"
FROM public.unemployment where "Stabr" in ('AZ','MI','WI','FL','PA','NC')
union
select
2016,"FIPStxt",
"Stabr",
"County",
" Unemployed_2016 ",
" Civilian_labor_force_2016 ","POP_COU"
FROM public.unemployment where "Stabr" in ('AZ','MI','WI','FL','PA','NC')
;''', create_engine_str)
unemployment_df.head()

,?column?,FIPStxt,Stabr,County,Unemployed_2000,Civilian_labor_force_2000,POP_COU
0,2004,26107,MI,Mecosta,"1,425","19,956",42798
1,2004,12033,FL,Escambia,"6,463","131,998",297619
2,2012,26093,MI,Livingston,"7,347","93,153",180967
3,2012,4007,AZ,Gila,"2,137","20,923",53597
4,2000,42103,PA,Pike,798,"21,255",57369


In [131]:
contrib_county_df["transaction_dt"].ast

dtype('O')

In [136]:
contrib_county_df['year']=contrib_county_df["transaction_dt"].astype("datetime64").dt.year
contrib_county_df['year'].value_counts()

1999    458
2003    139
2001    115
Name: year, dtype: int64

In [138]:
#vote_summary_df=votes_df.groupby(["year", "state", "state_po", "county","FIPS", "office", "party","candidatevotes"],as_index=False).sum()
contrib_county_df.groupby(["year","state_contrib","stcountyfp"],as_index=False).sum()

,year,state_contrib,stcountyfp,id,image_num,zip,transaction_amt,sub id
0,1999,az,4003.0,283550.0,4.005523e+10,171235.0,450.0,6.123840e+18
1,1999,az,4005.0,425311.0,2.180900e+11,258344.0,1700.0,9.185760e+18
2,1999,az,4012.0,141750.0,2.003514e+10,85346.0,500.0,3.061920e+18
3,1999,az,4013.0,7796700.0,3.471742e+12,4685026.0,30800.0,1.684056e+20
4,1999,az,4019.0,5245348.0,1.925951e+12,3171761.0,28950.0,1.132911e+20
...,...,...,...,...,...,...,...,...
116,2003,wi,55129.0,283456.0,4.604061e+10,109686.0,500.0,2.141640e+18
117,2003,wi,55131.0,141668.0,2.399207e+10,53029.0,1000.0,4.101620e+18
118,2003,wi,55133.0,708407.0,1.199596e+11,265491.0,2700.0,2.047950e+19
119,2003,wi,55135.0,141639.0,2.399137e+10,54742.0,500.0,4.071720e+18
